# 2. Feature Extraction - CSP

## Thach thuc

- Standard CSP: Overfitting voi du lieu it
- Accuracy: 70-75% (baseline)
- Gap train-test: 13-15%

## Y tuong

**Regularized CSP:** Them regularization vao covariance matrix

```
C_reg = (1-λ)C + λI
```

**Ly do:** Lambda can bang giua signal va noise, giam condition number

| Method | Accuracy | Overfitting Gap |
|--------|----------|----------------|
| Standard CSP | 72% | 13% |
| Regularized CSP | **77%** | **3%** |

**Ket qua:** +5% accuracy, giam overfitting 10%

## Code minh hoa (pseudo-code)

In [1]:
# Regularized CSP core idea
import numpy as np

# Standard CSP: C = average_covariance(X)
# Regularized CSP: C_reg = (1-λ)C + λI

def regularized_csp(C, lambda_reg=0.1):
    """Apply regularization to covariance matrix"""
    n = C.shape[0]
    I = np.eye(n)
    C_reg = (1 - lambda_reg) * C + lambda_reg * I
    return C_reg

# Example
C = np.random.randn(22, 22)
C = np.dot(C, C.T)  # Make positive definite

C_standard = C
C_regularized = regularized_csp(C, lambda_reg=0.1)

print(f"Condition number:")
print(f"  Standard:    {np.linalg.cond(C_standard):.1e}")
print(f"  Regularized: {np.linalg.cond(C_regularized):.1e}")
print(f"\nRegularization giam condition number → on dinh hon")

Condition number:
  Standard:    2.7e+03
  Regularized: 6.0e+02

Regularization giam condition number → on dinh hon
